In [3]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce
from time import time
from queue import PriorityQueue

def print_mat(mat):
    for row in mat:
        print(''.join(row))

files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [26]:
name = box.value
f = open(name, 'r')
lines = f.readlines()
lines = list(map(lambda l: l.rstrip(), lines))


# TODO: Parse the input

class Operation:
    def __init__(self, line):
        t = line.split()[3:]
        f = None
        if t[0] == '*':
            if t[1] == "old":
                self.operator = lambda x: x * x
            else:
                self.operator = lambda x: x * int(t[1])
        elif t[0] == '+':
            if t[1] == "old":
                self.operator = lambda x: x + x
            else:
                self.operator = lambda x: x + int(t[1])

    def call(self, old):
        return self.operator(old)


class Monkey:
    def __init__(self, items, operation, test):
        self.items = items
        self.operation = operation
        self.test = test
        self.inspected = 0


Monkeys = []
mod = 1
i = 0
while i < len(lines):
    items = list(map(int, lines[i+1][18:].split(", ")))
    operation = Operation(lines[i+2][13:])
    test = (int(lines[i+3][20:]),
            int(lines[i+4][29:]),
            int(lines[i+5][30:]))
    mod *= test[0]
    Monkeys.append(Monkey(items, operation, test))

    i += 7


print("Done parsing \"{name}\"".format(name=name))


Done parsing "in.txt"


### Part 1

In [11]:
M = deepcopy(Monkeys)

for r in range(20):
    for m in M:
        while len(m.items) > 0:
            item = m.items.pop(0)
            item = m.operation.call(item)
            m.inspected += 1
            item = item // 3
            x, a, b = m.test
            if item % x == 0:
                M[a].items.append(item)
            else:
                M[b].items.append(item)

    # print("After round {}:".format(r+1))
    # for i in range(len(M)):
        # m = M[i]
        # print("Monkey {}: {}".format(i, m.inspected))

I = list(map(lambda m: m.inspected, M))
I.sort()
I = np.array(I[::-1])
I[:2].prod()

10605

### Part 2

In [27]:
M = deepcopy(Monkeys)

F = []
for m in M:
    F.append(np.vectorize(lambda x: m.operation.operator(x) % mod))

G = []
for m in M:
    G.append(np.vectorize(lambda x: x % m.test[0]))

for r in range(10_000):
    for (m, f, g) in zip(M, F, G):
        if len(m.items) == 0:
            continue
        m.items = f(m.items)
        m.inspected += len(m.items)
        test = g(m.items)
        _, a, b = m.test
        M[a].items = np.append(M[a].items, m.items[test == 0])
        M[b].items = np.append(M[b].items, m.items[test != 0])
        m.items = []
        

I = list(map(lambda m: m.inspected, M))
I.sort()
I = np.array(I[::-1])
I[:2].prod()


14952185856